In [61]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import csv
import time
import pandas as pd
import re
import csv
import numpy as np

In [62]:
#Fonction  to start selenium driver

def start_driver():
    #Location of webdriver
    driver = webdriver.Chrome("C:\\Users\\nkeda\\Desktop\\MBD\\IESEG\\Training Materiels\\Financial Programming\\Indiviual Project\\chromedriver_win32\\chromedriver.exe")
    driver.maximize_window()
    return driver

def go_to_url(driver, url):
    driver.get(url)
    wait_for_page_to_load(driver)

def wait_for_page_to_load(driver, timeout_seconds=3):
    WebDriverWait(driver, timeout_seconds, 0.2)

In [63]:
driver = start_driver()
#upload the url contain in the csv file for the url over 30 days in December from 1st Dec to 31st
my_url_file = pd.read_csv("./urls_december.csv", header = None)
filename = "data_hotel_booking.csv" #File that will be generated after runing the code
with open(filename, "w", newline='') as csvfile:
    headers = ["Date","ID", "Hotel_Name", "Stars", "Reviews_score", "Reviews_classification", "Number_reviews", "Price", "Features", "Night", "Bed_configuration", "Location", "Location_Score","Lat_Long", "Taxe", "Web_page"]
    writer = csv.DictWriter(csvfile, fieldnames=headers) 
    writer.writeheader()
    for i in range(len(my_url_file)):
        my_url= my_url_file[0][i]
        go_to_url(driver, my_url)
        date = i+1
        time.sleep(2)
        page = 1
        while page in range(len(driver.find_elements_by_class_name("bui-pagination__item"))):

            try:

                time.sleep(2)
                hotels = driver.find_elements_by_class_name("sr_item")
                time.sleep(2)
                for hotel in hotels:
                    try:


                        hotel_id = hotel.get_attribute("data-hotelid")
                        hotel_name = hotel.find_element_by_class_name("sr-hotel__name").text
                        stars = hotel.get_attribute("data-class")
                        reviews_score = hotel.get_attribute("data-score")    
                        reviews_classification = hotel.find_element_by_class_name("bui-review-score__title").text    
                        number_reviews = hotel.find_element_by_class_name("bui-review-score__text").text
                        price = hotel.find_element_by_class_name("bui-price-display__value").text
                        features = hotel.find_element_by_class_name("room_link").text   
                        night = hotel.find_element_by_class_name("bui-price-display__label").text
                        bed_configuration = hotel.find_element_by_class_name("room_info").text        
                        location = hotel.find_element_by_class_name("sr_card_address_line").text
                        taxe_info = hotel.find_element_by_class_name("prd-taxes-and-fees-under-price").text
                        geo_info = hotel.find_element_by_class_name("bui-link").get_attribute("data-coords")
                        hotel_url = hotel.find_element_by_class_name("bui-link").get_attribute("href")
                        if len(hotel.find_elements_by_class_name("review-score-badge"))>0:
                            location_score = hotel.find_element_by_class_name("review-score-badge").text
                        else:
                            location_score = '0'
                    except:
                        pass          

                    writer.writerow({'Date':date,'ID':hotel_id,'Hotel_Name':hotel_name,'Stars':stars,'Reviews_score': reviews_score,'Reviews_classification':reviews_classification, 'Number_reviews':number_reviews,'Price':price , 'Features':features,'Night':night,'Bed_configuration':bed_configuration ,'Location':location, 'Location_Score':location_score, 'Lat_Long':geo_info, 'Taxe':taxe_info, 'Web_page':hotel_url})
                    time.sleep(2)

                time.sleep(5)
                if page <= (len(driver.find_elements_by_class_name("bui-pagination__item"))-1): #Condition to avoid clicking on the next button a the end of pagination
                    next_page = driver.find_element_by_class_name('paging-next')
                    driver.execute_script("arguments[0].click()", next_page)
                else:
                    pass
            except:
                pass


            page +=1
    csvfile.close()
    

In [ ]:
###REFERRENCE

#https://medium.com/analytics-vidhya/scraping-airbnb-fe1e895bd925
#https://www.scrapehero.com/scrape-property-data-from-booking-com-using-google-chrome/
#https://plotly.com/python/plotly-express/